In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import pandas as pd
import plotly.express as px

from spot.utils import (
    cst,
    read_file,
    write_file,
    seq_flatten,
    proj_root,
    tqdm,
)
from spot.type_env import (
    collect_annotations,
    MypyChecker,
    AnnotPath,
    mypy_checker,
    TypeInfEnv,
    TypeInfState,
    TypeInfAction,
    SelectAnnotations,
)
from spot.data_prepare import GitRepo

from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from typing import *

os.chdir(proj_root())

datadir = Path(os.getenv("datadir"))
repos_dir = datadir / "SPOT-data/repos"

useful_repos_path = proj_root() / "scripts" / "useful_repos.pkl"
with useful_repos_path.open("rb") as f:
    useful_repos: list[GitRepo] = pickle.load(f)

import random

random.seed(42)
random.shuffle(useful_repos)


In [3]:
# loading pre-trained model and tokenizer

model_dir = "./checkpoints/saved/SPOT-CodeT5-fine-tune/checkpoint-1500"

import torch
from transformers import RobertaTokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq
from transformers.models.t5 import T5ForConditionalGeneration

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer: RobertaTokenizer = RobertaTokenizer.from_pretrained(model_dir)
model: T5ForConditionalGeneration = T5ForConditionalGeneration.from_pretrained(model_dir).to(device)
max_target_length=128

/home/jiayi/Projects/SPOT/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [6]:
from spot.data_prepare import tokenize_masked, mask_type_annots, chunk_masked_code

test_code = """
# a, b, c are all ints.
def int_add(a: int, b: int, c: int) -> int:
    return a + b + c
"""

def run_model(code: str, num_beams=32):
    tks = tokenize_masked(mask_type_annots(code), tokenizer, device)
    loss = model.forward(**tks).loss
    dec = model.generate(
        tks['input_ids'],
        max_length=max_target_length,
        num_beams=num_beams,
        # do_sample=True,
    )[0]
    return {'loss': loss, 'generation': tokenizer.decode(dec)}

run_model(test_code)

{'loss': tensor(3.8123, device='cuda:0', grad_fn=<NllLossBackward0>),
 'generation': '<pad><s><extra_id_0>Union[int, Tuple[int, int]]<extra_id_1>Union[int, Tuple[int, int]]<extra_id_2>Union[int, Tuple[int, int]], Tuple[int, int]]<extra_id_3>Union[int, Tuple[int, int], Tuple[int, int]]</s>'}

In [13]:
test_code_2 = read_file(proj_root() / "src" / "spot" / "utils.py")
run_model(test_code_2, num_beams=1)

{'loss': tensor(0.3881, device='cuda:0', grad_fn=<NllLossBackward0>),
 'generation': '<pad><s><extra_id_0>str<extra_id_1>str<extra_id_2>None<extra_id_3>Path<extra_id_4>Sequence[T1]<extra_id_5>Sequence[T2]<extra_id_6>Sequence[str]<extra_id_7>Sequence[str]<extra_id_8>str</s>'}